In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
stroke = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
stroke = stroke.drop("id", axis=1) # dropping this as this is just for identification
stroke.head()

In [ ]:
stroke.info()

In [ ]:
cols= ["#C2C4E2","#EED4E5"]
sns.countplot(x= stroke["stroke"], palette= cols)

In [ ]:
stroke["gender"].value_counts()

In [ ]:
stroke["ever_married"].value_counts()

In [ ]:
stroke["work_type"].value_counts()

In [ ]:
stroke["Residence_type"].value_counts()

In [ ]:
stroke["smoking_status"].value_counts()

In [ ]:
stroke.describe()

In [ ]:
stroke.hist(bins= 50, figsize= (20,15))
plt.show()

In [ ]:
corr1 = stroke.corr()
corr1["stroke"].sort_values(ascending= False) 

In [ ]:
cat = (stroke.dtypes == "object")
cat_col = list(cat[cat].index)

print("Categorical variables")
print(cat_col)

In [ ]:
for i in cat_col:
  print(i, stroke[i].isnull().sum()) #dont want to take any chances

In [ ]:
stroke["bmi"].fillna(stroke["bmi"].median(), inplace=True) #as shown in info() this was only with less count

In [ ]:
stroke.info()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(transformers = [('cat', OneHotEncoder(), cat_col )], remainder='passthrough')
stroke = np.array(ct.fit_transform(stroke))

In [ ]:
stroke = pd.DataFrame(stroke)

In [ ]:
x = stroke.iloc[:, :-1].values
y = stroke.iloc[:,-1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier_log = LogisticRegression(random_state= 0)
classifier_log.fit(X_train, Y_train)

In [ ]:
y_pred = classifier_log.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred), 1), Y_test.reshape(len(Y_test), 1)), axis=1))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, y_pred)
print(cm)
print(accuracy_score(Y_test, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, Y_train)

In [ ]:
y_pred1 = classifier.predict(X_test)
cm = confusion_matrix(Y_test, y_pred1)
print(cm)
print(accuracy_score(Y_test, y_pred1))

In [ ]:
from sklearn.metrics import mean_squared_error
stroke_predictions = classifier.predict(X_train)
lin_mse = mean_squared_error(Y_train, stroke_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(classifier, X_train, Y_train, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

In [ ]:
from sklearn.model_selection import cross_val_score
score_l = cross_val_score(classifier_log, X_train, Y_train, scoring="neg_mean_squared_error", cv=10)
log_rmse_scores = np.sqrt(-scores)

In [ ]:
display_scores(log_rmse_scores)